## EMBEDDINGS

### embeddings extraction

In [95]:
import torch.nn as nn
import torch
import tensorflow as tf
import numpy as np
from src.data_preprocessing import datapreproces
dataset_name = "Beauty"
# data, num_batch, args, Features, CXTDict, UserFeatures = datapreproces(dataset_name)
# [user_train, user_valid, user_test, users_total_num, items_total_num] = data
index_list = [1,0]
users = [1,2]
seq_cxt = [CXTDict[(user, index_list[0])] for user in users]
seq_cxt = np.array(seq_cxt)
emb_items = nn.Embedding(Features.shape[0] + 1, 90, padding_idx=0, norm_type=0.01)

In [98]:
emb_items(torch.tensor(index_list))

tensor([[ 1.9471,  1.0317,  0.4846, -1.5641, -0.2368,  1.3876,  0.7347, -1.1573,
          0.2910,  1.3973, -1.6543, -0.5115,  0.3209, -0.5572, -0.1066,  1.9494,
          0.7083,  1.6507, -0.0610,  1.1979, -0.8466, -0.4442,  0.6275,  0.2956,
         -0.1482,  0.5476, -0.4038, -1.8332, -0.3552, -0.6705,  0.1667,  0.7754,
         -0.3378,  0.7090,  0.2868, -0.1666, -0.5058,  0.1329, -1.5623, -2.3170,
         -0.0599,  0.3859,  0.3412, -0.4067, -0.5002, -0.2441,  1.9822, -0.1858,
         -0.0726, -1.2569, -0.8453, -0.9545, -0.3456, -1.4746,  0.4829, -0.0375,
          0.0382,  1.2739, -1.0067,  1.4165, -0.4220, -0.9866, -0.0749,  0.5436,
         -1.9779, -0.9052, -0.4024,  2.0748,  0.2989, -0.0536,  0.4692,  1.2699,
          0.5089, -2.1045, -1.5540,  0.9174,  0.1993, -1.8354,  0.1797, -0.3698,
          0.4699, -0.8442,  0.5123, -0.3943,  0.6913,  0.5210, -0.5548,  0.1412,
          0.7923, -0.2882],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,


In [33]:
Features[0:2,:], seq_cxt

(array([[ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
        [33.99,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ]]),
 array([[ 0.92857143,  0.91666667,  0.77419355,  0.85714286,  0.89863014,
         11.75      ],
        [ 1.        ,  0.25      ,  0.61290323,  0.28571429,  0.21369863,
          3.        ]]))

In [52]:
emb = nn.Embedding(Features.shape[0], Features.shape[1])
emb._parameters['weight'] = Features
index_list = [1,0]
array = torch.tensor(emb._parameters['weight'][index_list])
# array = torch.tensor([[[1,2,3],[31,32,33]], [[11,12,13],[21,22,23]]])
print(f"{array.size()}")
input_shape = array.size()
array.view(-1, input_shape[-1])
print(f"{array.size()}")

torch.Size([2, 6507])
torch.Size([2, 6507])


In [94]:
x = torch.arange(15)
x = x.view(3, -1)
def get_mask1(tensor):
    return tensor.to(torch.bool) if tensor is not None else None
def get_mask(tensor):
    return torch.BoolTensor(tensor == 0) if tensor is not None else None
timeline_mask = torch.BoolTensor(x == 0)
# x *= ~timeline_mask
x.sum(dim=-1), get_mask1(x), ~get_mask(x).unsqueeze(-1).repeat(1, 1, 3)

(tensor([10, 35, 60]),
 tensor([[False,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]]),
 tensor([[[False, False, False],
          [ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]],
 
         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]],
 
         [[ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True],
          [ True,  True,  True]]]))

In [69]:
~timeline_mask.unsqueeze(-1)

tensor([[[False],
         [ True],
         [ True],
         [ True],
         [ True]],

        [[ True],
         [ True],
         [ True],
         [ True],
         [ True]],

        [[ True],
         [ True],
         [ True],
         [ True],
         [ True]]])

In [35]:
seq_feat = tf.nn.embedding_lookup(Features, index_list, name="seq_feat")

2023-07-26 10:26:53.528697: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 2982288240 exceeds 10% of free system memory.


In [40]:
seq_feat.shape, seq_cxt

(TensorShape([2, 6507]),
 array([[ 0.92857143,  0.91666667,  0.77419355,  0.85714286,  0.89863014,
         11.75      ],
        [ 1.        ,  0.25      ,  0.61290323,  0.28571429,  0.21369863,
          3.        ]]))

In [44]:
seq_feat_in = tf.concat([seq_feat, seq_cxt], -1)
hidden_units = 90
seq_feat_emb = tf.keras.layers.Dense(inputs=seq_feat_in, units=hidden_units * 5,
                                            activation=None,
                                            kernel_initializer=tf.random_normal_initializer(stddev=0.01),
                                            name="feat_emb")
# tf.concat([seq_in, seq_feat_emb], 2)

TypeError: ('Keyword argument not understood:', 'inputs')

### Embeddings Position

In [1]:
import math
from torch import nn

class SinPositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000, concat=False):
        """
        Inputs
            d_model - Hidden dimensionality of the input.
            max_len - Maximum length of a sequence to expect.
        """
        super().__init__()

        # Create matrix of [SeqLen, HiddenDim] representing the positional encoding for max_len inputs
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # even
        pe[:, 1::2] = torch.cos(position * div_term)  # odds
        pe = pe.unsqueeze(0)
        
        self.register_buffer('pe', pe, persistent=False)
        self.concat = concat
        if concat:
            self.encoding = nn.Linear(d_model * 2, d_model)

    def forward(self, x):
        if not self.concat:
            x = x + self.pe[:, :x.size(1)]
        else:
            x = torch.cat(x, self.pe[:, :x.size(1)])
            x = self.encoding(x)
        return x
max_len = 75
d_model = 10
concat = True

In [2]:
pe = torch.zeros(max_len, d_model)
position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
pe[:, 0::2] = torch.sin(position * div_term)  # even
pe[:, 1::2] = torch.cos(position * div_term)  # odds

NameError: name 'torch' is not defined

In [9]:
pe.shape, pe[-1]

(torch.Size([75, 10]),
 tensor([-0.9851,  0.1717, -0.7434,  0.6688,  0.9588, -0.2840,  0.2904,  0.9569,
          0.0467,  0.9989]))

In [10]:
pe = torch.arange(8).view(4,2)
pe2 = torch.flip(pe, [0])[0]

In [14]:
pe = pe.unsqueeze(-1)
pe.shape

torch.Size([1, 1, 4, 2, 1])

In [49]:
tensor = torch.cat([torch.zeros(2), torch.arange(8)])
t2 = tensor[tensor!=0]
t1 = tensor[tensor!=0].unsqueeze(-1)
t1.size(), t2.size()

(torch.Size([7, 1]), torch.Size([7]))

In [28]:
input_shape = t1.size()  # batch x length
seq = t1.view(-1, input_shape[-1])

In [50]:
attention_mask = tensor.clone()
attention_mask[attention_mask!=0] = 1

In [51]:
tensor

tensor([0., 0., 0., 1., 2., 3., 4., 5., 6., 7.])

In [52]:
attention_mask

tensor([0., 0., 0., 1., 1., 1., 1., 1., 1., 1.])

In [53]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

context = tokenizer(['It will rain in the', 'It will rain in the city but not here'], return_tensors='pt', padding=True)

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [54]:
context

{'input_ids': tensor([[ 101, 1135, 1209, 4458, 1107, 1103,  102,    0,    0,    0,    0],
        [ 101, 1135, 1209, 4458, 1107, 1103, 1331, 1133, 1136, 1303,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
head_mask = None

In [ ]:
p = (head_mask[idx] if head_mask is not None else None)

In [ ]:
import os
import sys
import time
from tqdm import tqdm

from src.data_preprocessing import datapreproces
from src.sampler import *

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

dataset_name =  "Beauty" # sys.argv[1]

data, num_batch, args, ItemFeatures, CXTDict, UserFeatures = datapreproces(dataset_name)
[user_train, user_valid, user_test, users_total_num, items_total_num] = data

sampler = WarpSampler(user_train, users_total_num, items_total_num, CXTDict,
                      args.cxt_size, batch_size=args.batch_size,
                      maxlen=args.maxlen, n_workers=3)

# model = Model(users_total_num, items_total_num, args, ItemFeatures, UserFeatures)

In [55]:
batch_size=args.batch_size
maxlen=args.maxlen
cxtsize = args.cxt_size
cxtdict=CXTDict

In [56]:
sampler.processors[0]

<Process(Process-1, started daemon)>

In [114]:
user = np.random.randint(1, users_total_num + 1)
while len(user_train[user]) <= 1: user = np.random.randint(1, users_total_num + 1)

In [115]:
user_train[user]

[44149, 1017, 2999]

In [184]:
def random_neq(l, r, s):
    """ random integer between l and r but avoiding s """
    t = np.random.randint(l, r)
    while t in s:
        t = np.random.randint(l, r)
    return t


In [126]:
# Sequence Padding
seq = np.zeros([maxlen], dtype=np.int32)
pos = np.zeros([maxlen], dtype=np.int32)
neg = np.zeros([maxlen], dtype=np.int32)
# Sequence Padding
seqcxt = np.zeros([maxlen, cxtsize], dtype=np.float32)
poscxt = np.zeros([maxlen, cxtsize], dtype=np.float32)
negcxt = np.zeros([maxlen, cxtsize], dtype=np.float32)

nxt = user_train[user][-1]
idx = maxlen - 1

ts = set(user_train[user])
for i in reversed(user_train[user][:-1]):
    print(i)

1017
44149


In [125]:
neg.shape,maxlen, cxtsize, nxt, idx,ts

((75,), 75, 6, 2999, 74, {1017, 2999, 44149})

In [212]:
[k for k,v in user_train.items() if len(v) < 6][:10]

[2, 5, 6, 7, 8, 9, 10, 12, 13, 14]

In [320]:
[(k,len(v)) for k,v in user_train.items() if len(v) > 75][:10]

[(113, 291),
 (114, 96),
 (163, 150),
 (177, 115),
 (189, 155),
 (226, 181),
 (311, 80),
 (324, 82),
 (396, 107),
 (422, 202)]

In [ ]:
import os
import sys
import time
from tqdm import tqdm

from src.data_preprocessing import datapreproces
from src.sampler import *

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

dataset_name = "Beauty"

data, num_batch, args, ItemFeatures, CXTDict, UserFeatures = datapreproces(dataset_name)
[user_train, user_valid, user_test, users_total_num, items_total_num] = data

sampler = WarpSampler(user_train, users_total_num, args, items_total_num, CXTDict)
model = Model(items_total_num, args, ItemFeatures)


 The dataset Beauty contains 52204 users and 57289 items in total
average sequence length: {5.63}
ItemFeatures DF dimensions (57290, 6507)
iterations_num 1469053


NameError: name 'Model' is not defined

## LOSSS

In [6]:
import torch.nn as nn
class BPRLoss(nn.Module):
    """ BPRLoss, based on Bayesian Personalized Ranking

    Args:
        - gamma(float): Small value to avoid division by zero

    Shape:
        - Pos_score: (N)
        - Neg_score: (N), same shape as the Pos_score
        - Output: scalar.
    """

    def __init__(self, gamma=1e-10):
        super(BPRLoss, self).__init__()
        self.gamma = gamma

    def forward(self, pos_score, neg_score):
        loss = -torch.log(self.gamma + torch.sigmoid(pos_score - neg_score)).mean()
        return loss
loss = BPRLoss()
loss(torch.tensor(list(range(100))), torch.tensor(list(range(100))))

tensor(0.6931)